# Imports

In [44]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
# from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import seaborn as sn
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
import nltk
from sklearn.metrics import roc_auc_score, f1_score
import seaborn as sn
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error #add rmse
from data import merged


# Reading files and making it regressionable

In [45]:
df_train = pd.read_csv('data/merged/merged_cleaned_sentiment_train.csv').drop(['pos','neg','neu', 'compound'], axis = 1)
df_val = pd.read_csv('data/merged/merged_cleaned_sentiment_validation.csv').drop(['pos','neg','neu', 'compound'], axis = 1)
df_test = pd.read_csv('data/merged/merged_cleaned_sentiment_test.csv').drop(['pos','neg','neu', 'compound'], axis = 1)

In [46]:
df_train = df_train[['danceability', 'energy', 'instrumentalness', 'valence','mode', 'y_valence', 'y_arousal']]
df_val = df_val[['danceability', 'energy', 'instrumentalness', 'valence','mode', 'y_valence', 'y_arousal']]
df_test = df_test[['danceability', 'energy', 'instrumentalness', 'valence','mode','y_valence', 'y_arousal']]


In [47]:
df_train = pd.concat([df_train, pd.read_csv('data/lyrics/lyrics_features_train.csv').iloc[:, :-200]], axis=1)
df_val = pd.concat([df_val, pd.read_csv('data/lyrics/lyrics_features_val.csv').iloc[:, :-200]], axis=1)
df_test = pd.concat([df_test, pd.read_csv('data/lyrics/lyrics_features_test.csv').iloc[:, :-200]], axis=1)

### This was when we used all audio features

df_train = pd.concat([df_train, pd.get_dummies(df_train.key, drop_first = True, prefix = 'key')], axis=1)
df_val = pd.concat([df_val, pd.get_dummies(df_val.key, drop_first = True, prefix = 'key')], axis=1)
df_test = pd.concat([df_test, pd.get_dummies(df_val.key, drop_first = True, prefix = 'key')], axis=1)

df_train = df_train.drop(['Unnamed: 0', 'artist', 'trackname', 'id', 'time_signature', 'lyrics', 'key','lyrics_cleaned' ], axis=1)
df_val = df_val.drop(['Unnamed: 0', 'artist', 'trackname', 'id', 'time_signature', 'lyrics', 'key', 'lyrics_cleaned' ], axis=1)
df_test = df_test.drop(['Unnamed: 0', 'artist', 'trackname', 'id', 'time_signature', 'lyrics', 'key', 'lyrics_cleaned'], axis=1)

In [48]:
df_train = df_train.dropna()
df_val = df_val.dropna()
df_test = df_test.dropna()

In [49]:
df_train.columns

Index(['danceability', 'energy', 'instrumentalness', 'valence', 'mode',
       'y_valence', 'y_arousal', 'Unnamed: 0', 'pos', 'neg',
       ...
       'tfidf_pca_91', 'tfidf_pca_92', 'tfidf_pca_93', 'tfidf_pca_94',
       'tfidf_pca_95', 'tfidf_pca_96', 'tfidf_pca_97', 'tfidf_pca_98',
       'tfidf_pca_99', 'tfidf_pca_100'],
      dtype='object', length=112)

In [50]:
# function to get cross validation scores
def get_cv_scores(model, X_train, y_train):
    scores = cross_val_score(model,
                             X_train,
                             y_train,
                             cv=5,
                             scoring='r2')
    
    print('CV Mean: ', np.mean(scores))
    print('STD: ', np.std(scores))

# Splitting the data into X and y

In [51]:
#     train set
X_train = df_train.drop(['y_valence', 'y_arousal'], axis=1).values
y_train_valence = df_train.y_valence.values 
y_train_arousal = df_train.y_arousal.values
    
#     validation set
X_val = df_val.drop(['y_valence', 'y_arousal'], axis=1).values
y_val_valence = df_val.y_valence.values 
y_val_arousal = df_val.y_arousal.values 

#      test set
X_test = df_test.drop(['y_valence', 'y_arousal'], axis=1).values
y_test_valence = df_test.y_valence.values 
y_test_arousal = df_test.y_arousal.values

In [52]:
LinearRegression().fit(y_train_arousal.reshape(-1, 1), y_train_valence.reshape(-1, 1)).score(y_train_arousal.reshape(-1, 1), y_train_valence.reshape(-1, 1))

0.054508397801767416

In [53]:
def do_regression(X, y_1, y_2, X_validation, y_1_validation, y_2_validation):
    

    # parameters
    param_grid = {'fit_intercept':[True,False], 'positive':[True, False]}
    
    # Initialize model for Grid search
    lr_val = LinearRegression()
    lr_arou = LinearRegression()
    
    # Grid search
    clf_vale = GridSearchCV(lr_val, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)
    
    clf_arou = GridSearchCV(lr_arou, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)

    # Print best results on training data    
    clf_vale.fit(X, y_1)
    clf_arou.fit(X, y_2)
    
    # Print best results on training data
    # add new lines to separate rows
    print()
    print("Best parameter for Valence (CV score=%0.3f):" % clf_vale.best_score_)
    print(clf_vale.best_params_)
    
    print()
    print("Best parameter for Arousal (CV score=%0.3f):" % clf_arou.best_score_)
    print(clf_arou.best_params_)
    print()


    #Initialize models with best parameters
    lr_val_top = LinearRegression(fit_intercept=clf_vale.best_params_['fit_intercept'],  
                                    positive = clf_vale.best_params_['positive'])
    lr_arou_top = LinearRegression(fit_intercept=clf_arou.best_params_['fit_intercept'],  
                                    positive = clf_arou.best_params_['positive'])

    # get cross val scores for models 
    get_cv_scores(lr_val_top, X, y_1)
    get_cv_scores(lr_arou_top, X, y_2)


    #fit optimal models to train data 
    lr_val_fit = lr_val_top.fit(X, y_1)
    lr_arou_fit = lr_arou_top.fit(X, y_2)
    
    # validation scores 
    r2_validation_valence = lr_val_fit.score(X_validation, y_1_validation)
    r2_validation_arousal = lr_arou_fit.score(X_validation, y_2_validation)
    
    print()
    print(f'Validation score for Valence: {r2_validation_valence}')
    print(f'Validation score for Arousal: {r2_validation_arousal}')
    
    return clf_vale.best_params_, clf_arou.best_params_

In [54]:
def do_forest_regression(X, y_1, y_2, X_validation, y_1_validation, y_2_validation):
    
    # Initialize models
    rf_val = RandomForestRegressor(random_state=0)
    rf_arou = RandomForestRegressor(random_state=0)
    
    param_grid = { 
    'n_estimators': [100, 500],
    'max_depth' : [5,10, 15]
    }

    # Grid search
    clf_vale = GridSearchCV(rf_val, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)
    
    clf_arou = GridSearchCV(rf_arou, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)

    # Print best results on training data    
    clf_vale.fit(X, y_1)
    clf_arou.fit(X, y_2)

    # Print best results on training data
    print()
    print("Best parameter for Valence (CV score=%0.3f):" % clf_vale.best_score_)
    print(clf_vale.best_params_)
    
    print()
    print("Best parameter for Arousal (CV score=%0.3f):" % clf_arou.best_score_)
    print(clf_arou.best_params_)


    #Initialize models with best parameters
    rf_val_top = RandomForestRegressor(n_estimators = clf_vale.best_params_['n_estimators'], 
                                        max_depth = clf_vale.best_params_['max_depth'], random_state=0)
    rf_arou_top = RandomForestRegressor(n_estimators = clf_arou.best_params_['n_estimators'], 
                                        max_depth = clf_arou.best_params_['max_depth'], random_state=0)
    

    # get cross val scores
    get_cv_scores(rf_val_top, X, y_1)
    get_cv_scores(rf_arou_top, X, y_2)

    rf_val_fit = rf_val_top.fit(X, y_1)
    rf_arou_fit = rf_arou_top.fit(X, y_2)
    
    r2_validation_valence = rf_val_fit.score(X_validation, y_1_validation)
    r2_validation_arousal = rf_arou_fit.score(X_validation, y_2_validation)

    print()
    print(f'Validation score for Valence: {r2_validation_valence}')
    print(f'Validation score for Arousal: {r2_validation_arousal}')
    
    
    return clf_vale.best_params_, clf_arou.best_params_

In [55]:
def do_svr(X, y_1, y_2, X_validation, y_1_validation, y_2_validation):

    # Normalization
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X_validation = scaler.fit_transform(X_validation)

    # Train model
    svr_val = SVR()
    svr_arou = SVR()
    
    param_grid = {'kernel' : ('linear', 'rbf', 'poly'),
                  'C' : [1,5,10]
                 }

    # Grid search
    clf_vale = GridSearchCV(svr_val, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)
    
    clf_arou = GridSearchCV(svr_arou, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)

    # Print best results on training data    
    clf_vale.fit(X, y_1)
    clf_arou.fit(X, y_2)
    
    # Print best results on training data
    # add new lines to separate rows
    print()
    print("Best parameter for Valence (CV score=%0.3f):" % clf_vale.best_score_)
    print(clf_vale.best_params_)
    
    print()
    print("Best parameter for Arousal (CV score=%0.3f):" % clf_arou.best_score_)
    print(clf_arou.best_params_)
    print()


    # Train model
    svr_val_top = SVR(kernel = clf_vale.best_params_['kernel'], C = clf_vale.best_params_['C'])
    svr_arou_top = SVR(kernel = clf_arou.best_params_['kernel'], C = clf_arou.best_params_['C'])


    # get cross val scores
    get_cv_scores(svr_val_top, X, y_1)
    get_cv_scores(svr_arou_top, X, y_2)

    #fit
    svr_val_fit = svr_val_top.fit(X,y_1)
    svr_arou_fit = svr_arou_top.fit(X, y_2)
    
    r2_validation_valence = svr_val_fit.score(X_validation, y_1_validation)
    r2_validation_arousal = svr_arou_fit.score(X_validation, y_2_validation)
    
    print()
    print(f'Validation score for Valence: {r2_validation_valence}')
    print(f'Validation score for Arousal: {r2_validation_arousal}')


    return clf_vale.best_params_, clf_arou.best_params_

In [56]:
def do_mlp(X, y_1, y_2, X_validation, y_1_validation, y_2_validation):

    # Normalization
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X_validation = scaler.fit_transform(X_validation)

    # Initialize model
    mlp_val = MLPRegressor(random_state = 2)
    mlp_arou = MLPRegressor(random_state = 2)
    
    param_grid = {'hidden_layer_sizes':[(5), (10), (15), (5,5), (10,10), (15,15), (5,5,5), (10,10,10), (15,15,15)], 
    'max_iter':[500, 1000, 2000, 2500]}

    # Grid search
    clf_vale = GridSearchCV(mlp_val, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)
    
    clf_arou = GridSearchCV(mlp_arou, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)


    # Print best results on training data    
    clf_vale.fit(X, y_1)
    clf_arou.fit(X, y_2)
    
    # Print best results on training data
    # add new lines to separate rows
    print()
    print("Best parameter for Valence (CV score=%0.3f):" % clf_vale.best_score_)
    print(clf_vale.best_params_)
    
    print()
    print("Best parameter for Arousal (CV score=%0.3f):" % clf_arou.best_score_)
    print(clf_arou.best_params_)
    print()


    #Train model with best params
    mlp_val_top = MLPRegressor(hidden_layer_sizes=clf_vale.best_params_['hidden_layer_sizes'], max_iter=clf_vale.best_params_['max_iter'], random_state = 2)
    mlp_arou_top = MLPRegressor(hidden_layer_sizes=clf_arou.best_params_['hidden_layer_sizes'], max_iter=clf_arou.best_params_['max_iter'], random_state = 2)

    # get cross val scores 
    get_cv_scores(mlp_val_top, X, y_1)
    get_cv_scores(mlp_arou_top, X, y_2)

    mlp_val_fit = mlp_val_top.fit(X, y_1)
    mlp_arou_fit = mlp_arou_top.fit(X, y_2)
    
    r2_validation_valence = mlp_val_fit.score(X_validation, y_1_validation)
    r2_validation_arousal = mlp_arou_fit.score(X_validation, y_2_validation)
    
    print()
    print(f'Validation score for Valence: {r2_validation_valence}')
    print(f'Validation score for Arousal: {r2_validation_arousal}')

    
    return clf_vale.best_params_, clf_arou.best_params_

### Random Forest Regression - Optimization

In [57]:
val_par_rf, arou_par_rf = do_forest_regression(X_train, y_train_valence, y_train_arousal, X_val, y_val_valence, y_val_arousal)


Fitting 5 folds for each of 6 candidates, totalling 30 fits


KeyboardInterrupt: 

    Fitting 5 folds for each of 6 candidates, totalling 30 fits
    Fitting 5 folds for each of 6 candidates, totalling 30 fits

    Best parameter for Valence (CV score=0.185):
    {'max_depth': 10, 'n_estimators': 500}

    Best parameter for Arousal (CV score=0.199):
    {'max_depth': 10, 'n_estimators': 500}
    CV Mean:  0.18477634266575188
    STD:  0.010064224970315037
    CV Mean:  0.1987682194468258
    STD:  0.025811738331714372

    Validation score for Valence: 0.17296757628631831
    Validation score for Arousal: 0.24101399592965878

#### Test Set - RF

In [ ]:
rf_val = RandomForestRegressor(n_estimators = 100, 
                                        max_depth = 5, random_state=0)
rf_arou = RandomForestRegressor(n_estimators = 100, 
                                        max_depth = 5, random_state=0)


In [ ]:
print(rf_val.fit(X_train, y_train_valence).score(X_test, y_test_valence))
print(rf_arou.fit(X_train, y_train_arousal).score(X_test, y_test_arousal))

0.21563447935080082
0.2031321009223268


### Linear Regression - optimization

In [ ]:
val_par, arou_par = do_regression(X_train, y_train_valence, y_train_arousal, X_val, y_val_valence, y_val_arousal)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Fitting 5 folds for each of 4 candidates, totalling 20 fits

Best parameter for Valence (CV score=0.192):
{'fit_intercept': True, 'positive': False}

Best parameter for Arousal (CV score=0.165):
{'fit_intercept': False, 'positive': False}

CV Mean:  0.19156200216250585
STD:  0.012919083070775876
CV Mean:  0.1646334689838204
STD:  0.02101493280205238

Validation score for Valence: 0.18598784431685378
Validation score for Arousal: 0.19393910784235824


    Fitting 5 folds for each of 4 candidates, totalling 20 fits
    Fitting 5 folds for each of 4 candidates, totalling 20 fits

    Best parameter for Valence (CV score=0.193):
    {'fit_intercept': True, 'positive': False}

    Best parameter for Arousal (CV score=0.164):
    {'fit_intercept': True, 'positive': True}

    CV Mean:  0.19263909417367236
    STD:  0.01423911801666168
    CV Mean:  0.16387005569169152
    STD:  0.013789304326383702

    Validation score for Valence: 0.18365452891704448
    Validation score for Arousal: 0.18418929034767606

#### LR Test Set 

In [ ]:
lr_val = LinearRegression(fit_intercept=val_par['fit_intercept'],  
                                    positive = val_par['positive'])
lr_arou = LinearRegression(fit_intercept=arou_par['fit_intercept'],  
                                    positive = arou_par['positive'])

In [ ]:
print(lr_val.fit(X_train, y_train_valence).score(X_test, y_test_valence))
print(lr_arou.fit(X_train, y_train_arousal).score(X_test, y_test_arousal))


0.23522504334112415
0.18979073068866747


### Support Vector Regression - optimization

In [ ]:
val_par_svr, arou_par_svr = do_svr(X_train, y_train_valence, y_train_arousal, X_val, y_val_valence, y_val_arousal)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits

Best parameter for Valence (CV score=0.171):
{'C': 1, 'kernel': 'linear'}

Best parameter for Arousal (CV score=0.157):
{'C': 1, 'kernel': 'linear'}

CV Mean:  0.17060915752620148
STD:  0.016451994401807103
CV Mean:  0.15681334193429142
STD:  0.021609634567641076

Validation score for Valence: 0.14513143825250574
Validation score for Arousal: 0.17697077330583544


#### Test Score - SVR

In [ ]:
svr_val = SVR(kernel = val_par_svr['kernel'], C = val_par_svr['C'])
svr_arou = SVR(kernel = arou_par_svr['kernel'], C = arou_par_svr['C'])

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
print(svr_val.fit(X_train, y_train_valence).score(X_test, y_test_valence))
print(svr_arou.fit(X_train, y_train_arousal).score(X_test, y_test_arousal))

0.21211076671348172
0.17257770158185026


### Multilayer Perceptron Regression - optimization

In [ ]:
val_par_mlp, arou_par_mlp = do_mlp(X_train, y_train_valence, y_train_arousal, X_val, y_val_valence, y_val_arousal)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Fitting 5 folds for each of 36 candidates, totalling 180 fits

Best parameter for Valence (CV score=0.200):
{'hidden_layer_sizes': (5, 5), 'max_iter': 500}

Best parameter for Arousal (CV score=0.175):
{'hidden_layer_sizes': (5, 5, 5), 'max_iter': 500}

CV Mean:  0.19950950041566404
STD:  0.015591862192069152
CV Mean:  0.17458027826593903
STD:  0.021119598144282097

Validation score for Valence: 0.18311694698515346
Validation score for Arousal: 0.2009258008111775


    Fitting 5 folds for each of 36 candidates, totalling 180 fits
    Fitting 5 folds for each of 36 candidates, totalling 180 fits

    Best parameter for Valence (CV score=0.198):
    {'hidden_layer_sizes': (5, 5, 5), 'max_iter': 500}

    Best parameter for Arousal (CV score=0.177):
    {'hidden_layer_sizes': (5, 5, 5), 'max_iter': 500}

    CV Mean:  0.19816558523065714
    STD:  0.013616324243766003
    CV Mean:  0.17746596775912477
    STD:  0.022757705925352777

    Validation score for Valence: 0.16755552530343532
    Validation score for Arousal: 0.2115436506271312
    

### Test Set - MLP 

In [ ]:
mlp_val = MLPRegressor(hidden_layer_sizes=val_par_mlp['hidden_layer_sizes'], max_iter=val_par_mlp['max_iter'], random_state = 2)
mlp_arou = MLPRegressor(hidden_layer_sizes=arou_par_mlp['hidden_layer_sizes'], max_iter=arou_par_mlp['max_iter'], random_state = 2)


In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
X_train.shape

(7596, 110)

In [ ]:
print(mlp_val.fit(X_train, y_train_valence).score(X_test, y_test_valence))
print(mlp_arou.fit(X_train, y_train_arousal).score(X_test, y_test_arousal))

0.2380058059037995
0.19579112679579103


### Feature Selection

In [ ]:
from sklearn.feature_selection import RFE


In [ ]:

for i in [5,10,15,20,25]:

    print(i, 'FEATURES')
    estimator = LinearRegression(fit_intercept= True, positive= True)
    selector = RFE(estimator, n_features_to_select=i, step=1)
    selector = selector.fit(X_train, y_train_valence)

    estimator2 = LinearRegression(fit_intercept= True, positive= True)
    selector2 = RFE(estimator2, n_features_to_select=i, step=1)
    selector_arou = selector2.fit(X_train, y_train_arousal)

    lr_valence = LinearRegression(fit_intercept= True, positive= True)
    lr_arousal = LinearRegression(fit_intercept= True, positive= True)
    get_cv_scores(estimator, X_train[:,selector.support_], y_train_valence)
    get_cv_scores(estimator, X_train[:,selector_arou.support_], y_train_arousal)

    print()

5 FEATURES
CV Mean:  0.16450630521856338
STD:  0.013886221462250624
CV Mean:  0.16240066093528122
STD:  0.016278244786631514

10 FEATURES
CV Mean:  0.17711296964776696
STD:  0.012236324459514152
CV Mean:  0.16629974906378792
STD:  0.013766632635808412

15 FEATURES
CV Mean:  0.1858072293248198
STD:  0.012142036198196548
CV Mean:  0.16707987220261172
STD:  0.012195808791533468

20 FEATURES
CV Mean:  0.18750891541565198
STD:  0.012395120883625089
CV Mean:  0.1710413548962561
STD:  0.012456230700412279

25 FEATURES
CV Mean:  0.18757014047620452
STD:  0.011272469226122862
CV Mean:  0.1706936926551956
STD:  0.012209891108903371



In [ ]:
# 20 features
estimator = LinearRegression(fit_intercept= True, positive= True)
selector = RFE(estimator, n_features_to_select=20, step=1)
selector = selector.fit(X_train, y_train_valence)

estimator2 = LinearRegression(fit_intercept= True, positive= True)
selector2 = RFE(estimator2, n_features_to_select=20, step=1)
selector_arou = selector2.fit(X_train, y_train_arousal)

In [ ]:
mlp_valence = MLPRegressor(hidden_layer_sizes= (10,10), max_iter=500)
mlp_arousal = MLPRegressor(hidden_layer_sizes= 5, max_iter=500)
mlp_val_fit = mlp_valence.fit(X_train[:,selector.support_], y_train_valence)
mlp_arou_fit = mlp_arousal.fit(X_train[:,selector_arou.support_], y_train_arousal)

r2_validation_valence = mlp_val_fit.score(X_val[:,selector.support_], y_val_valence)
r2_validation_arousal = mlp_arou_fit.score(X_val[:,selector_arou.support_], y_val_arousal)

In [ ]:

print()
print(f'Validation score for Valence: {r2_validation_valence}')
print(f'Validation score for Arousal: {r2_validation_arousal}')


Validation score for Valence: -0.10761250799503541
Validation score for Arousal: -0.22313442641479098


In [ ]:
df_train.columns[selector.support_]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 112 but corresponding boolean dimension is 110

In [ ]:
df_train.columns[selector2.support_]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 112 but corresponding boolean dimension is 110

In [ ]:
df_train.head()

,danceability,energy,instrumentalness,valence,mode,y_valence,y_arousal,Unnamed: 0,pos,neg,...,tfidf_pca_91,tfidf_pca_92,tfidf_pca_93,tfidf_pca_94,tfidf_pca_95,tfidf_pca_96,tfidf_pca_97,tfidf_pca_98,tfidf_pca_99,tfidf_pca_100
0,0.320,0.798,0.25100,0.407,0,-0.682725,0.316758,0,0.142,0.174,...,0.015783,-0.032266,0.006463,-0.003021,-0.035761,-0.011026,-0.018885,0.017530,-0.040245,0.018009
1,0.434,0.600,0.45100,0.753,0,0.373325,-0.923151,1,0.276,0.000,...,0.007425,0.006276,-0.057914,0.006029,0.000979,-0.023851,-0.010867,-0.012319,-0.035166,-0.003449
2,0.530,0.270,0.00000,0.515,0,1.071901,0.846830,2,0.159,0.095,...,0.018198,-0.019298,-0.058143,0.030132,-0.020258,-0.017809,0.049364,-0.004692,0.012285,-0.009272
3,0.479,0.528,0.00128,0.185,0,-1.056572,0.736206,3,0.099,0.037,...,0.007089,0.009937,-0.003824,0.027660,-0.017931,0.004531,-0.013273,-0.006385,-0.000347,0.001329
4,0.639,0.893,0.00816,0.966,1,1.541257,1.704165,4,0.231,0.106,...,0.003749,0.004186,-0.020362,-0.016839,0.005286,-0.018892,0.018980,0.016430,0.000353,-0.001808


In [ ]:
do_mlp(X_train, y_train_valence, y_train_arousal, X_val, y_val_valence, y_val_arousal)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Fitting 5 folds for each of 36 candidates, totalling 180 fits

Best parameter for Valence (CV score=0.200):
{'hidden_layer_sizes': (5, 5), 'max_iter': 500}

Best parameter for Arousal (CV score=0.175):
{'hidden_layer_sizes': (5, 5, 5), 'max_iter': 500}

CV Mean:  0.19950950041566404
STD:  0.015591862192069152
CV Mean:  0.17458027826593903
STD:  0.021119598144282097

Validation score for Valence: 0.18311694698515346
Validation score for Arousal: 0.2009258008111775


({'hidden_layer_sizes': (5, 5), 'max_iter': 500},
 {'hidden_layer_sizes': (5, 5, 5), 'max_iter': 500})